In [ ]:
import json

with open("./data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

docs[0]['content']
#len(docs[0].content)
#embeddings = get_embeddings_in_batches(docs)

In [1]:
import pandas as pd

# 로컬 JSON 파일 읽기
df = pd.read_json("./data/documents.jsonl", lines=True)
df.tail()


,docid,src,content
4267,ae28101b-a42e-45b7-b24b-4ea0f1fb2d50,ko_ai2_arc__ARC_Challenge__train,비뇨기계와 순환계는 혈액이 신장을 통과하면서 폐기물과 물이 제거될 때 관여하는 두 ...
4268,eb727a4f-29c7-4d0c-b364-0e67de1776e9,ko_ai2_arc__ARC_Challenge__train,로봇은 현대 산업에서 많은 역할을 수행할 수 있습니다. 그러나 로봇 사용의 중대한 ...
4269,0c8c0086-c377-4201-81fa-25159e5435a7,ko_mmlu__human_sexuality__test,"월경은 여성의 생리주기에 따라 발생하는 현상으로, 에스트로겐과 프로게스테론 수치의 ..."
4270,06da6a19-ec78-404e-9640-9fc33f63c6a2,ko_ai2_arc__ARC_Challenge__train,식물이 내뿜는 가스는 산소입니다. 식물은 광합성 과정을 통해 태양 에너지를 이용하여...
4271,03c36d5e-c711-4dc2-b4db-aaeb94d86395,ko_mmlu__computer_security__test,"버퍼 오버런은 개발자들이 만든 널리 퍼져 있는 앱의 코딩 오류로써, 공격자가 시스템..."


In [ ]:
df.iloc[0].content

length = df['content'].str.len()
min = df['content'].str.len().min()
mask = (length == min)
df['content'][mask]

In [ ]:
from ast import Str
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("")
model = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

result = chain.invoke()

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

# 1. 프롬프트 템플릿 정의
SCIENCE_5W1H_TEMPLATE = """
과학문서 분석 전문가로서 육하원칙에 따라 체계적으로 요약해주세요.
    - 문서의 길이가 100자 이내라면 원문 그대로 사용한다.
    - 모든문장에는 육하원칙 6가지 요소가 반드시 포함되어야 한다.
    - 각 문장에는 원문에 포함된 핵심키워드 2개이상이 사용되어야 한다.

문서: {document}

응답 형식은 반드시 다음과 같이 해주세요:
**1차 요약**: [문서의 길이가 100자 이내라면 원문 그대로 사용 또는 400자 내외 요약]
"""


# 2. 프롬프트 객체 생성
science_prompt = PromptTemplate(
    input_variables=["document"],
    template=SCIENCE_5W1H_TEMPLATE
)

# 3. LLM 모델 선택 (Ollama 또는 OpenAI)
# llm = Ollama(model="llama3.1:8b")  # 로컬 모델
llm = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")

# 4. 출력 파서 (문자열)
output_parser = StrOutputParser()

# 5. LCEL 체인 구성
science_summary_chain = (
    science_prompt 
    | llm 
    | output_parser
)

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama


first_summary = """
과학문서 분석 전문가로서 육하원칙에 따라 체계적으로 요약해주세요.
    - 아래 문서의 길이가 100자 이내라면 원문 그대로 사용한다.
    - 전체 요약길이는 문서길이의 절반내외로 한다.
    - 모든문장에는 육하원칙 6가지 요소가 반드시 포함되어야 한다.
    - 각 문장에는 원문에 포함된 핵심키워드 2개이상이 사용되어야 한다.
        
문서: {document}

응답 형식은 반드시 다음과 같이 해주세요:
요약:"""

second_summary = """
과학문서 분석 전문가로서 육하원칙에 따라 체계적으로 요약해주세요.
    - 아래 문서의 길이가 100자 이내라면 원문 그대로 사용한다.
    - 전체 요약길이는 문서길이의 절반내외로 한다. 
    - 모든문장에는 육하원칙 6가지 요소가 반드시 포함되어야 한다.
    - 각 문장에는 원문에 포함된 핵심키워드 2개이상이 사용되어야 한다.

문서: {first_summary}

응답 형식은 반드시 다음과 같이 해주세요:
요약:"""


first_prompt = PromptTemplate(
    input_variables=["document"],
    template=first_summary
)


second_prompt = PromptTemplate(
    input_variables=["document"],
    template=second_summary

)

# 체인 생성 - 1차 요약 후 바로 2차 요약으로 연결
two_step_chain = (
    #{"text": RunnablePassthrough()}
    first_prompt 
    | llm 
    | StrOutputParser()
    | (lambda x: print(f"1차 요약: {x}") or x) 
    | (lambda x: {"first_summary": x})
    | second_prompt
    | llm
    | StrOutputParser()
)



In [25]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore

class ScienceRAG:
    def __init__(self):
        self.embeddings = HuggingFaceEmbeddings(
                                    model_name="BAAI/bge-m3",
                                    model_kwargs={"device": "cuda"} ,
                                    encode_kwargs={"normalize_embeddings": True})
        self.vectorstore = FAISS.from_texts(["초기화용 더미 텍스트"], self.embeddings)
        """  # 임베딩 차원 계산
        dimension = len(self.embeddings.embed_query("test"))
    
        # 코사인 유사도를 위한 IndexFlatIP 생성
        index = faiss.IndexFlatIP(dimension)
    
        # 빈 FAISS 벡터 스토어 생성
        self.vectorstore = FAISS(
            embedding_function=self.embeddings,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
            normalize_L2=True  # 벡터 정규화로 코사인 유사도 구현
        ) """

        # 요약 체인
        #self.summary_chain = science_summary_chain
        self.summary_chain = two_step_chain
    
    def add_document(self, doc_id: str, document: str):
        """문서를 요약하여 벡터DB에 저장"""
        
        # 1. 문서 요약
        summary_result = self.summary_chain.invoke({"document": document})
        
        # 2. 검색용 요약을 벡터DB에 저장
        self.vectorstore.add_texts(
            [summary_result],
            metadatas=[{"doc_id": doc_id, "original_doc": document}]
        )
        
        return summary_result
    
    def search(self, query: str, k: int = 3):
        """요약된 내용으로 검색"""
        results = self.vectorstore.similarity_search(query, k=k)
        return results

In [26]:
# 사용 예시
science_rag = ScienceRAG()

for index, data in df.iterrows():
    if index == 5: break
    
    summary = science_rag.add_document(data.docid, data.content)
    print("*"*50)
    print("*"*50)
    print(data.content)
    print(summary)

summary = science_rag.add_document(df.iloc[1339].docid, df.iloc[1339].content)
print("*"*50)
print("*"*50)
print(df.iloc[1339].content)
print(summary)    

#search_results = science_rag.search("당뇨병 신약 효과")

1차 요약: 요약: 건강한 사람은 에너지 균형을 평형 상태로 유지하기 위해 1-2주간 식단과 운동을 조절해야 하며, 이 과정에서 에너지 섭취와 에너지 소비의 수학적 동등성을 확보해야 한다. 식단은 영양가 있는 식품과 적절한 칼로리 섭취를 포함하고, 운동은 에너지 소비를 촉진하고 근육을 강화시켜 에너지 균형을 유지하는 데 기여한다. 이러한 조절을 통해 비만이나 영양 실조와 같은 건강 문제를 예방할 수 있다.
**************************************************
**************************************************
건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.
요약: 건강한 사람은 1-2주간 식단 조절과 운동을 통해 에너지 균형을 유지해야 하며, 이 과정에서 에너지 섭취와 소비의 수학적 동등성을 확보해야 한다. 영양가 있는 식품과 적절한 칼로리 섭취는 식단 조절의 핵심이며, 운동은 에너지 소비 촉진과 근육 강화에 기여한다. 이러한 조절은 비만 예방과 영양 실조 예방에 효과적이다.
1차 요약: 요약: 수소 분자는 수소, 산소, 질소 가스 혼합물에서 가장 가볍고 작은 원자로 구성되어 있어 평균 속도가 가장 빠릅니다. 이는 주어진 온도에서 수소 분자가 다른 분자들보다 더

In [27]:
query = "마이애미파랑나비"
docs = science_rag.vectorstore.similarity_search_with_score(query, k=4)
docs

[(Document(id='e93b12cc-e053-494b-a457-d29254a79122', metadata={'doc_id': '910107a6-2a42-41a2-b337-fbf22d6440fe', 'original_doc': '마이애미파랑나비는 남부 플로리다에서 멸종 위기에 처한 종입니다. 이 나비의 개체수 감소를 초래했을 가능성이 가장 높은 요인은 주택 건설 증가입니다. 남부 플로리다 지역에서는 최근 몇 년간 주택 건설이 급증하였고, 이로 인해 서식지가 감소하고 개체들의 서식 환경이 파괴되었습니다. 마이애미파랑나비는 특정한 식물에 의존하여 번식하고 생존하는데, 주택 건설로 인해 이 식물들의 서식지가 감소하였기 때문에 개체수가 감소한 것으로 추정됩니다. 이러한 주택 건설 증가는 마이애미파랑나비뿐만 아니라 다른 생물종들에게도 영향을 미치고 있으며, 생태계의 균형을 위해 적절한 조치가 필요합니다. 주택 건설 증가를 제한하거나 대안적인 서식지를 마련하는 등의 방안이 고려되어야 합니다.'}, page_content='요약: 마이애미파랑나비는 남부 플로리다에서 주택 건설 증가로 인한 서식지 파괴로 인해 멸종 위기에 처했으며, 특정 식물 의존 생태계 붕괴로 인해 개체수 감소가 발생했다.'),
  np.float32(0.84816396)),
 (Document(id='c464305c-33a3-409c-b578-582a1abaa8ee', metadata={}, page_content='초기화용 더미 텍스트'),
  np.float32(1.232441)),
 (Document(id='b64b53ae-ed1d-45e2-a311-980d59caab63', metadata={'doc_id': '74f22819-1a8e-4646-8a9d-13323de8cdb8', 'original_doc': '비버는 나무를 베고, 덤불과 관목을 모아 강과 개울에 댐을 만드는 것으로 알려져 있습니다. 이러한 댐은 비버의 서식지를 만들어주고 다양한 동물들에게도 이점을 제공합니다. 그러나 최근에는